# 0.6_ss_全探索
- ちゃんと写経する

In [26]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline

In [4]:
# fibonacci
def fib(n: int):
    if n <= 1:
        return 1
    return fib(n-1) + fib(n-2)

In [13]:
%timeit fib(20)

3.8 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
memo = np.zeros(30+1)
def fib_dp(n: int):
    if n <= 1: 
        return 1
    if memo[n] == 0:
        memo[n] = fib(n-1) + fib(n-2)
    return memo[n]

In [16]:
%timeit memo = np.zeros(30+1); fib_dp(20)

2.04 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
n = 4
a = [1,2,4,7]
k = 13

In [22]:
def dfs(i, sum_):
    if i == n:
        return sum_ == k
    if dfs(i+1, sum_):
        return True
    if dfs(i+1, sum_ + a[i]):
        return True
    return False

In [23]:
dfs(0, 0)

True

## Lake counting

In [89]:
from io import StringIO

lakes = '''
W.........W.
.WWW......WW.
......WWW...
WW......W..
....WW.WWW...
........WW..
........W..
W.........
.W........
..W.......
.W........
'''

f = StringIO(lakes)
df = pd.read_table(f, header=None)
df.columns = ['lakes']
df_split = df.lakes.str.split('', expand=True)
df_lakes = df_split.loc[:, df_split.isna().sum(0)==0]
df_lakes = df_lakes.loc[:, (df_lakes == '').sum() == 0]

N, M = df_lakes.shape

print(f"{df_lakes}")
print(f"N = {N}, M = {M}")

   1  2  3  4  5  6  7  8  9  10
0   W  .  .  .  .  .  .  .  .  .
1   .  W  W  W  .  .  .  .  .  .
2   .  .  .  .  .  .  W  W  W  .
3   W  W  .  .  .  .  .  .  W  .
4   .  .  .  .  W  W  .  W  W  W
5   .  .  .  .  .  .  .  .  W  W
6   .  .  .  .  .  .  .  .  W  .
7   W  .  .  .  .  .  .  .  .  .
8   .  W  .  .  .  .  .  .  .  .
9   .  .  W  .  .  .  .  .  .  .
10  .  W  .  .  .  .  .  .  .  .
N = 11, M = 10


In [95]:
def dfs(x, y):
    df_lakes.iloc[x, y] = '.'
    dd = [-1, 0, 1]
    for (dx, dy) in itertools.product(dd, dd):
        nx = x + dx
        ny = y + dy
        # out of the field
        if (nx < 0) | (ny < 0) | (nx >= N) | (ny >= M):
            continue
        # if water
        if df_lakes.iloc[nx, ny] == 'W':
            dfs(nx, ny)

In [96]:
res = 0
for i in range(N):
    for j in range(M):
        if df_lakes.iloc[i, j] == 'W':
            dfs(i, j)
            res += 1
print(f"There are {res} lakes!")

There are 5 lakes!


## Maze

In [108]:
maze = '''
#S###### #
       # #
 # ## ## # 
 #       
## ## ####
   #
 ####### #
    #
 #### ### 
    #   G#
'''

In [204]:
f = StringIO(maze)
df = pd.read_table(f, header=None)
df.columns = ['maze']
df_maze = df.maze.str.split('', expand=True)
df_maze.fillna(' ', inplace=True)
df_maze = df_maze.iloc[:, 1:10]


In [206]:
df_maze

,1,2,3,4,5,6,7,8,9
0,#,S,#,#,#,#,#,#,
1,,,,,,,,#,
2,,#,,#,#,,#,#,
3,,#,,,,,,,
4,#,#,,#,#,,#,#,#
5,,,,#,,,,,
6,,#,#,#,#,#,#,#,
7,,,,,#,,,,
8,,#,#,#,#,,#,#,#
9,,,,,#,,,,G


In [111]:
moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [117]:
d = pd.DataFrame(np.ones(df_maze.shape)*np.inf)
d

,0,1,2,3,4,5,6,7,8
0,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,inf,inf,inf,inf,inf,inf,inf,inf,inf
9,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [226]:
que = [] 

In [220]:
import pdb

In [248]:
def bfs(df_maze):
    N, M = df_maze.shape
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    d = pd.DataFrame(np.ones(df_maze.shape)*np.inf)

    # starts
    sx, sy = [tup[0] for tup in np.where(df_maze == 'S')]
    gx, gy = [tup[0] for tup in np.where(df_maze == 'G')]

    # que
    que = [(sx, sy)]
    d.iloc[sx, sy] = 0

    while len(que) > 0:
        px, py = que.pop(0)
        if (px == gx) & (py == gy):
            print("finished", px, gx, py, gy)
            return print(d); d.iloc[gx, gy]

        # move
        for dx, dy in moves:
            nx = px + dx
            ny = py + dy
            if (nx >= 0) and (ny >= 0) and (nx < N) and (ny < M) and (df_maze.iloc[nx, ny] != "#"):
                que.append((nx, ny))
                d.iloc[nx, ny] = min(d.iloc[nx, ny], d.iloc[px, py] + 1)

    return print(d); d.iloc[gx, gy] # return inf


In [252]:
from IPython.display import display, clear_output

In [256]:
# def bfs(df_maze):
N, M = df_maze.shape
moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
d = pd.DataFrame(np.ones(df_maze.shape)*np.inf)

# starts
sx, sy = [tup[0] for tup in np.where(df_maze == 'S')]
gx, gy = [tup[0] for tup in np.where(df_maze == 'G')]

# que
que = [(sx, sy)]
d.iloc[sx, sy] = 0
i = 0

while len(que) > 0:
    i += 1
    clear_output(wait=True)
    display(f"{i}th iteration")
    
    px, py = que.pop(0)
    if (px == gx) & (py == gy):
        print("finished", px, gx, py, gy)
        break
        # return print(d); d.iloc[gx, gy]

    # move
    for dx, dy in moves:
        nx = px + dx
        ny = py + dy
        if (nx >= 0) and (ny >= 0) and (nx < N) and (ny < M) and (df_maze.iloc[nx, ny] != "#"):
            # 既存のほうがベター
            if d.iloc[nx, ny] < d.iloc[px, py] + 1:
                continue
            que.append((nx, ny))
            d.iloc[nx, ny] = min(d.iloc[nx, ny], d.iloc[px, py] + 1)

# return print(d); d.iloc[gx, gy] # return inf

'74th iteration'

finished 9 9 8 8


In [257]:
d

,0,1,2,3,4,5,6,7,8
0,inf,0.0,inf,inf,inf,inf,inf,inf,13.0
1,2.0,1.0,2.0,3.0,4.0,5.0,6.0,inf,12.0
2,3.0,inf,3.0,inf,inf,6.0,inf,inf,11.0
3,4.0,inf,4.0,5.0,6.0,7.0,8.0,9.0,10.0
4,inf,inf,5.0,inf,inf,8.0,inf,inf,inf
5,8.0,7.0,6.0,inf,10.0,9.0,10.0,11.0,12.0
6,9.0,inf,inf,inf,inf,inf,inf,inf,13.0
7,10.0,11.0,12.0,13.0,inf,17.0,16.0,15.0,14.0
8,11.0,inf,inf,inf,inf,18.0,inf,inf,inf
9,12.0,13.0,14.0,15.0,inf,19.0,20.0,21.0,22.0


In [258]:
df_maze

,1,2,3,4,5,6,7,8,9
0,#,S,#,#,#,#,#,#,
1,,,,,,,,#,
2,,#,,#,#,,#,#,
3,,#,,,,,,,
4,#,#,,#,#,,#,#,#
5,,,,#,,,,,
6,,#,#,#,#,#,#,#,
7,,,,,#,,,,
8,,#,#,#,#,,#,#,#
9,,,,,#,,,,G
